In [1]:
import cupy as cp
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import scipy.stats as st
import pickle
import optuna
import re
import gc

from lightgbm import LGBMClassifier

from sklearn import metrics
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.impute import SimpleImputer
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.metrics import roc_auc_score
from sklearn.datasets import make_moons
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import mutual_info_classif
from sklearn.experimental import enable_iterative_imputer

from keras.layers import Dense, Dropout
from keras.models import Sequential

from six import StringIO

from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import math
from scipy import stats
import numpy as np
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow import feature_column
from tensorflow.keras import layers

from tqdm import tqdm

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
from sklearn.metrics import log_loss
from xgboost.sklearn import XGBClassifier

from Modules import Modules

from Baseline import Baseline

%matplotlib inline

print("OK\n")

OK



In [2]:
app_train_origin =  pd.read_csv('./home-credit-default-risk/application_train.csv')
app_test_origin =  pd.read_csv('./home-credit-default-risk/application_test.csv')
app_train_origin.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
app_train = app_train_origin.copy()
app_test = app_test_origin.copy()
app_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
display(app_train['DAYS_EMPLOYED'].value_counts())
print("正の値の割合: {:.4f}".format((app_train['DAYS_EMPLOYED']>0).mean()))
print("正の値の個数: {}".format((app_train['DAYS_EMPLOYED']>0).sum()))

 365243    55374
-200         156
-224         152
-230         151
-199         151
           ...  
-13961         1
-11827         1
-10176         1
-9459          1
-8694          1
Name: DAYS_EMPLOYED, Length: 12574, dtype: int64

正の値の割合: 0.1801
正の値の個数: 55374


In [5]:
app_train['DAYS_EMPLOYED'] = app_train['DAYS_EMPLOYED'].replace(365243, np.nan)

In [6]:
# 特徴量1: 総所得金額を世帯人数で除した値
app_train['INCOME_div_PERSON'] = app_train['AMT_INCOME_TOTAL'] / app_train['CNT_FAM_MEMBERS']

# 特徴量2: 総所得額を就労期間で除した値
app_train['INCOME_div_EMPLOYED'] = app_train['AMT_INCOME_TOTAL'] / app_train['DAYS_EMPLOYED']

# 特徴量3: 外部スコアの平均
app_train['EXT_SOURCE_mean'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
    
app_train['EXT_SOURCE_max'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].max(axis=1)

app_train['EXT_SOURCE_min'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].min(axis=1)

app_train['EXT_SOURCE_std'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)

app_train['EXT_SOURCE_count'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].notnull().sum(axis=1)

# 特徴量4: 就労期間を年齢で除した値
app_train['DAYS_EMPLOYED_div_BERTH'] = app_train['DAYS_EMPLOYED'] / app_train['DAYS_BIRTH']

# 特徴量5: 年金支払金額を所得金額で除した値
app_train['DYAS_EMPLOYED_div_BIRTH'] = app_train['DAYS_EMPLOYED'] / app_train['DAYS_BIRTH']

# 特徴量6: 年金支払額を借入金で除した値
app_train['ANNUITY_div_INCOME'] = app_train['AMT_ANNUITY'] / app_train['AMT_CREDIT']

In [7]:
x_train = app_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = app_train['TARGET']
id_train = app_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=='O':
        x_train[col] = x_train[col].astype('category')

In [8]:
baseline = Baseline('SK_ID_CURR')

params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.05
    ,'num_leaves': 32
    ,'n_estimators': 100000
    ,'random_state':123
    ,'importance_type': 'gain'
}

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 130) (61503, 130)
[100]	training's auc: 0.788109	valid_1's auc: 0.760759
[200]	training's auc: 0.816418	valid_1's auc: 0.763517
[300]	training's auc: 0.837598	valid_1's auc: 0.764063
[400]	training's auc: 0.855146	valid_1's auc: 0.764164
[auc] tr:0.847577, va0.7644
-------------------- 1 --------------------
(246009, 130) (61502, 130)
[100]	training's auc: 0.788535	valid_1's auc: 0.763435
[200]	training's auc: 0.816357	valid_1's auc: 0.767263
[300]	training's auc: 0.837623	valid_1's auc: 0.767251
[auc] tr:0.819819, va0.7675
-------------------- 2 --------------------
(246009, 130) (61502, 130)
[100]	training's auc: 0.787378	valid_1's auc: 0.763712
[200]	training's auc: 0.816398	valid_1's auc: 0.766646
[300]	training's auc: 0.837717	valid_1's auc: 0.767428
[400]	training's auc: 0.855193	valid_1's auc: 0.76748
[auc] tr:0.853514, va0.7676
-------------------- 3 --------------------
(246009, 130) (61502, 130)
[100]	training's auc: 0.7878

In [119]:
imp.sort_values("imp", ascending=False)[:10]

,col,imp,imp_std
44,EXT_SOURCE_mean,113403.491162,1106.065693
10,ANNUITY_div_INCOME,22633.310916,1055.984794
112,ORGANIZATION_TYPE,21132.114234,2070.062321
41,EXT_SOURCE_3,10975.516564,1213.466831
23,DAYS_BIRTH,7303.002100,1202.272651
45,EXT_SOURCE_min,7192.419023,547.311260
39,EXT_SOURCE_1,6523.085309,641.490575
0,AMT_ANNUITY,6075.770187,689.556328
2,AMT_GOODS_PRICE,5719.635271,349.597120
46,EXT_SOURCE_std,5035.130996,778.073568


In [121]:
# nullに置き換え
app_test['DAYS_EMPLOYED'] = app_test['DAYS_EMPLOYED'].replace(365243, np.nan)

# 特徴量1: 総所得金額を世帯人数で除した値
app_test['INCOME_div_PERSON'] = app_test['AMT_INCOME_TOTAL'] / app_test['CNT_FAM_MEMBERS']

# 特徴量2: 総所得額を就労期間で除した値
app_test['INCOME_div_EMPLOYED'] = app_test['AMT_INCOME_TOTAL'] / app_test['DAYS_EMPLOYED']

# 特徴量3: 外部スコアの平均
app_test['EXT_SOURCE_mean'] = app_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
    
app_test['EXT_SOURCE_max'] = app_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].max(axis=1)

app_test['EXT_SOURCE_min'] = app_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].min(axis=1)

app_test['EXT_SOURCE_std'] = app_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)

app_test['EXT_SOURCE_count'] = app_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].notnull().sum(axis=1)

# 特徴量4: 就労期間を年齢で除した値
app_test['DAYS_EMPLOYED_div_BERTH'] = app_test['DAYS_EMPLOYED'] / app_test['DAYS_BIRTH']

# 特徴量5: 年金支払金額を所得金額で除した値
app_test['DYAS_EMPLOYED_div_BIRTH'] = app_test['DAYS_EMPLOYED'] / app_test['DAYS_BIRTH']

# 特徴量6: 年金支払額を借入金で除した値
app_test['ANNUITY_div_INCOME'] = app_test['AMT_ANNUITY'] / app_test['AMT_CREDIT']

# データセット
x_test = app_test.drop(columns=['SK_ID_CURR'])
id_test = app_test[['SK_ID_CURR']]

for col in x_test.columns:
    if x_test[col].dtype=='O':
        x_test[col] = x_test[col].astype('category')

In [122]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [123]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_featureEngineering1.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.029423
1,100005,0.116901
2,100013,0.019938
3,100028,0.044851
4,100038,0.180004


In [124]:
#app_train_origin = pd.read_csv('./home-credit-default-risk/exports/app_train.csv')
#app_test_origin =  pd.read_csv('./home-credit-default-risk/exports/app_test.csv')
bureau_origin = pd.read_csv('./home-credit-default-risk/bureau.csv')
bureau_balance_origin = pd.read_csv('./home-credit-default-risk/bureau_balance.csv')
pos_cash_balance_origin = pd.read_csv('./home-credit-default-risk/POS_CASH_balance.csv')
credit_card_balance_origin = pd.read_csv('./home-credit-default-risk/credit_card_balance.csv')
previous_application_origin = pd.read_csv('./home-credit-default-risk/previous_application.csv')
installments_payments_origin = pd.read_csv('./home-credit-default-risk/installments_payments.csv')
# custom data
#app_train_loan_from_hc_only_cash_origin = pd.read_csv('./home-credit-default-risk/datasets/app_train_loan_from_hc_only_cash.csv')
#app_test_loan_from_hc_only_cash_origin = pd.read_csv('./home-credit-default-risk/datasets/app_test_loan_from_hc_only_cash.csv')
#app_train_1000_mean_origin = pd.read_csv('./home-credit-default-risk/exports/train_hc_only_cash_installment_payment_ratio_1000_mean_mean.csv')
#app_test_1000_mean_origin = pd.read_csv('./home-credit-default-risk/exports/test_hc_only_cash_installment_payment_ratio_1000_mean_mean.csv')
#app_train_max_installment_ratio_origin = pd.read_csv('./home-credit-default-risk/exports/train_hc_annuity_to_max_installment_ratio.csv')
#app_test_max_installment_ratio_origin = pd.read_csv('./home-credit-default-risk/exports/test_hc_annuity_to_max_installment_ratio.csv')

In [125]:
#app_train = app_train_origin.copy()
#app_test = app_test_origin.copy()
bureau = bureau_origin.copy()
bureau_balance = bureau_balance_origin.copy()
pos_cash_balance = pos_cash_balance_origin.copy()
credit_card_balance = credit_card_balance_origin.copy()
previous_application = previous_application_origin.copy()
installments_payments = installments_payments_origin.copy()
#app_train_1000_mean = app_train_1000_mean_origin.copy()
#app_test_1000_mean = app_test_1000_mean_origin.copy()
#app_train_max_installment_ratio = app_train_max_installment_ratio_origin.copy()
#app_test_max_installment_ratio = app_test_max_installment_ratio_origin.copy()
pos_cash_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0
